In [ ]:
from moviepy import VideoFileClip
import os

def cut_highlight(video_path, highlights, output_dir="highlights"):
    os.makedirs(output_dir, exist_ok=True)
    clips = []
    for idx, highlight in enumerate(highlights):
        try:
            clip = VideoFileClip(video_path).subclip(highlight['start'], highlight['end'])

            # ⚡ Make it vertical (crop center)
            if clip.w > clip.h:  # if horizontal
                clip = clip.crop(
                    width=clip.h * 9/16,  # make it 9:16
                    height=clip.h,
                    x_center=clip.w/2,
                    y_center=clip.h/2
                )

            # Resize to full vertical HD (1080x1920)
            clip = clip.resize(height=1920, width=1080)

            output_path = os.path.join(output_dir, f"highlight_{idx+1}.mp4")
            clip.write_videofile(output_path, codec="libx264")
            clips.append(output_path)
        except Exception as e:
            print(f"❌ Clip creation failed: {e}")
    return clips


In [ ]:
from pytube import YouTube

def download_video(video_url, output_path="video.mp4"):
    yt = YouTube(video_url)
    stream = yt.streams.filter(progressive=True, file_extension="mp4").order_by('resolution').desc().first()
    stream.download(filename=output_path)
    print(f"✅ Downloaded video to {output_path}")
    return output_path


In [ ]:
import ollama
import json

def find_interesting_segments(transcription_result):
    text = transcription_result["text"]

    prompt = f"""
Given this transcription:

{text}

- Identify 3 of the most interesting moments (important sentences, funny moments, powerful messages).
- For each moment, estimate a start time and end time (in seconds).
- Provide a short description for each.

Respond exactly in this JSON format:
[
  {{ "start": 15, "end": 30, "text": "A touching story about friendship" }},
  {{ "start": 90, "end": 110, "text": "Funny mistake during a trip" }},
  ...
]
Only output the JSON array, no extra explanation.
    """

    response = ollama.chat(
        model='llama3',
        messages=[{"role": "user", "content": prompt}]
    )

    try:
        highlights = json.loads(response['message']['content'])
        return highlights
    except json.JSONDecodeError:
        print("❌ Error decoding highlights JSON.")
        return []


In [ ]:
from pytube import YouTube
from moviepy import VideoFileClip
import whisper
import os
import json
import yt_dlp

def download_video(video_url, output_path="videos"):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': f"{output_path}/%(title)s.%(ext)s",
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            return ydl.prepare_filename(info)
    except Exception as e:
        print(f"Download failed: {e}")
        return None

def extract_audio(video_path, audio_path="audio.wav"):
    try:
        clip = VideoFileClip(video_path)
        clip.audio.write_audiofile(audio_path)
        return audio_path
    except Exception as e:
        print(f"❌ Error extracting audio: {e}")
        return None

def transcribe_audio(audio_path):
    try:
        model = whisper.load_model("small")
        result = model.transcribe(audio_path)
        return result
    except Exception as e:
        print(f"❌ Error transcribing audio: {e}")
        return None
import ollama
# These functions need to be defined or imported
def find_interesting_segments(transcription_result):
    text = transcription_result["text"]

    prompt = f"""
Given this transcription:

{text}

- Identify 3 of the most interesting moments (important sentences, funny moments, powerful messages).
- For each moment, estimate a start time and end time (in seconds).
- Provide a short description for each.

Respond exactly in this JSON format:
[
  {{ "start": 15, "end": 30, "text": "A touching story about friendship" }},
  {{ "start": 90, "end": 110, "text": "Funny mistake during a trip" }},
  ...
]
Only output the JSON array, no extra explanation.
    """

    response = ollama.chat(
        model='llama3',
        messages=[{"role": "user", "content": prompt}]
    )

    try:
        highlights = json.loads(response['message']['content'])
        return highlights
    except json.JSONDecodeError:
        print("❌ Error decoding highlights JSON.")
        return []

import subprocess
import os
from datetime import timedelta

def cut_highlight(video_path, highlights, output_dir="highlights"):
    """
    Create highlight clips using FFmpeg (faster and more reliable than MoviePy)
    """
    os.makedirs(output_dir, exist_ok=True)
    created_clips = []
    
    for i, highlight in enumerate(highlights, 1):
        output_path = os.path.join(output_dir, f"highlight_{i}.mp4")
        
        # Convert seconds to HH:MM:SS.microseconds
        start_time = str(timedelta(seconds=highlight['start']))
        end_time = str(timedelta(seconds=highlight['end']))
        
        try:
            subprocess.run([
                'ffmpeg',
                '-y',  # Overwrite without asking
                '-i', video_path,
                '-ss', start_time,
                '-to', end_time,
                '-c:v', 'libx264',  # H.264 codec
                '-c:a', 'aac',      # AAC audio
                '-movflags', '+faststart',  # For web playback
                '-loglevel', 'error',  # Only show errors
                output_path
            ], check=True)
            
            created_clips.append(output_path)
            print(f"✅ Created highlight: {output_path}")
            
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to create clip {i}: {e}")
        except Exception as e:
            print(f"❌ Unexpected error with clip {i}: {e}")
    
    return created_clips

def generate_title(text):
    prompt = f"""
Given this highlight text:

{text}

Generate a short, exciting YouTube short title using emojis and curiosity. Only return the title, no explanation.
    """
    response = ollama.chat(
        model='llama3',
        messages=[{"role": "user", "content": prompt}]
    )
    return response['message']['content'].strip()

def main():
    video_url = input("Paste YouTube URL here: ")
    video_path = download_video(video_url)
    
    if not video_path:
        return
        
    audio_path = extract_audio(video_path)
    if not audio_path:
        return
        
    transcription_result = transcribe_audio(audio_path)
    if not transcription_result:
        return

    highlights = find_interesting_segments(transcription_result)

    clips = cut_highlight(video_path, highlights)

    print("\n✨ Generated Titles:")
    for idx, highlight in enumerate(highlights):
        title = generate_title(highlight['text'])
        print(f"  Clip {idx+1}: {title}")

    print("\n✅ Done! Check the 'highlights' folder for your clips.")

if __name__ == "__main__":
    main()

In [ ]:
import subprocess
import os
import json
import cv2
import numpy as np
from datetime import timedelta
from PIL import Image, ImageDraw, ImageFont
import ollama
from pytube import YouTube
import whisper
import random
import textwrap
import yt_dlp

# Configuration
CONFIG = {
    "output_dir": "output",
    "font_path": "arial.ttf",  # Replace with your font file
    "bg_music": "background.mp3",  # Optional background music
    "ollama_model": "llama3",  # Or "mistral"/"phi3"
    "whisper_model": "medium",
    "target_resolution": (1080, 1920)  # Shorts/Reels format
}

def setup_directories():
    """Create necessary folders"""
    os.makedirs(CONFIG["output_dir"], exist_ok=True)
    os.makedirs(os.path.join(CONFIG["output_dir"], "clips"), exist_ok=True)
    os.makedirs(os.path.join(CONFIG["output_dir"], "assets"), exist_ok=True)

def download_video(video_url, output_path="videos"):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': f"{output_path}/%(title)s.%(ext)s",
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            return ydl.prepare_filename(info)
    except Exception as e:
        print(f"Download failed: {e}")
        return None

def extract_audio(video_path):
    """Extract high-quality audio for transcription"""
    audio_path = os.path.join(CONFIG["output_dir"], "audio.wav")
    subprocess.run([
        'ffmpeg', '-y', '-i', video_path,
        '-vn', '-acodec', 'pcm_s16le',
        '-ar', '44100', '-ac', '2',
        '-loglevel', 'error',
        audio_path
    ], check=True)
    return audio_path

def transcribe_with_whisper(audio_path):
    """Transcribe with word-level timestamps"""
    model = whisper.load_model(CONFIG["whisper_model"])
    result = model.transcribe(audio_path, word_timestamps=True)
    return result


def analyze_content(transcript):
    """Use Ollama to find highlights and generate metadata"""
    prompt = f"""
    Analyze this transcript and identify 3-5 most engaging 8-15 second clips for Shorts/Reels.
    Return JSON with list of highlights, each containing EXACTLY these keys:
    - start (float): start time in seconds
    - end (float): end time in seconds
    - text (str): the spoken words
    - reason (str): why this is engaging

    Example:
    {{
        "highlights": [
            {{
                "start": 45.2,
                "end": 53.7,
                "text": "This changed everything...",
                "reason": "Emotional peak"
            }}
        ]
    }}

    Transcript:
    {json.dumps(transcript['segments'], indent=2)}
    """
    
    try:
        response = ollama.generate(
            model=CONFIG["ollama_model"],
            prompt=prompt,
            format="json"
        )
        result = json.loads(response["response"])
        
        # Extract highlights with validation
        highlights = result.get('highlights', [])
        if not isinstance(highlights, list):
            highlights = [highlights] if isinstance(highlights, dict) else []
        
        # Validate each highlight
        valid_highlights = []
        for h in highlights:
            if all(key in h for key in ['start', 'end', 'text', 'reason']):
                valid_highlights.append({
                    'start': float(h['start']),
                    'end': float(h['end']),
                    'text': str(h['text']),
                    'reason': str(h['reason'])
                })
        
        return valid_highlights or simple_highlight_detection(transcript)
    except Exception as e:
        print(f"⚠️ AI analysis failed: {e}")
        return simple_highlight_detection(transcript)


def simple_highlight_detection(transcript):
    """Fallback that returns valid highlights"""
    return [
        {
            'start': max(0, s['start'] - 1),
            'end': min(s['end'] + 1, transcript['segments'][-1]['end']),
            'text': s.get('text', 'Interesting moment'),
            'reason': "Auto-selected segment"
        }
        for s in transcript['segments'][:3]  # Take first 3 segments
        if 'start' in s and 'end' in s
    ]
def create_vertical_clip(base_video, highlight, clip_num):
    """Generate Shorts/Reels with effects"""
    output_path = os.path.join(CONFIG["output_dir"], "clips", f"clip_{clip_num}.mp4")
    temp_path = os.path.join(CONFIG["output_dir"], "clips", f"temp_{clip_num}.mp4")
    
    # 1. Cut the clip with 0.5s padding
    start = max(0, highlight['start'] - 0.5)
    end = highlight['end'] + 0.5
    
    # 2. Create vertical crop with smart framing
    subprocess.run([
        'ffmpeg', '-y',
        '-ss', str(start),
        '-to', str(end),
        '-i', base_video,
        '-vf', f"crop=ih*9/16:ih,scale={CONFIG['target_resolution'][0]}:{CONFIG['target_resolution'][1]},zoompan=z='min(zoom+0.0015,1.2)':d=1:x='if(gte(zoom,1.2),x,x+1)':y='y'",
        '-c:v', 'libx264', '-preset', 'fast', '-crf', '22',
        '-an',  # Temporary remove audio
        '-loglevel', 'error',
        temp_path
    ], check=True)
    
    # 3. Add dynamic subtitles
    final_path = add_animated_subtitles(temp_path, highlight['text'], start, end)
    
    # 4. Mix with background music
    return add_background_music(final_path, clip_num)

def add_animated_subtitles(video_path, text, start, end):
    """Burn stylish animated subtitles into video"""
    output_path = video_path.replace('.mp4', '_subtitled.mp4')
    
    # 1. Create temporary SRT file (more compatible than ASS)
    srt_path = os.path.join(CONFIG["output_dir"], "subtitles.srt")
    with open(srt_path, 'w', encoding='utf-8') as f:
        f.write(f"1\n{timedelta(seconds=0)} --> {timedelta(seconds=end-start)}\n{text}\n")
    
    # 2. Generate subtitles with fallback options
    try:
        # Try with style first
        subprocess.run([
            'ffmpeg', '-y',
            '-i', video_path,
            '-vf', f"subtitles='{srt_path}':force_style='FontName=Arial,FontSize=24,PrimaryColour=&HFFFFFF&,OutlineColour=&H000000&,Alignment=10'",
            '-c:v', 'libx264',
            '-preset', 'fast',
            '-crf', '22',
            '-loglevel', 'error',
            output_path
        ], check=True, shell=True)
    except subprocess.CalledProcessError:
        # Fallback to simple subtitles
        try:
            subprocess.run([
                'ffmpeg', '-y',
                '-i', video_path,
                '-vf', f"drawtext=text='{text}':fontfile={CONFIG['font_path']}:fontsize=24:fontcolor=white:box=1:boxcolor=black@0.5:x=(w-text_w)/2:y=h-text_h-50",
                '-c:v', 'libx264',
                '-preset', 'fast',
                '-loglevel', 'error',
                output_path
            ], check=True, shell=True)
        except Exception as e:
            print(f"❌ Subtitle fallback failed: {e}")
            return video_path  # Return original if all fails
    
    return output_path

def add_background_music(video_path, clip_num):
    """Mix with background music at low volume"""
    if not os.path.exists(CONFIG["bg_music"]):
        return video_path  # Skip if no music
    
    final_path = video_path.replace('.mp4', '_final.mp4')
    
    subprocess.run([
        'ffmpeg', '-y',
        '-i', video_path,
        '-i', CONFIG["bg_music"],
        '-filter_complex', '[0:a]volume=1.0[a0];[1:a]volume=0.3[a1];[a0][a1]amix=inputs=2:duration=shortest',
        '-c:v', 'copy',
        '-loglevel', 'error',
        '-shortest',
        final_path
    ], check=True)
    
    return final_path

def generate_social_metadata(highlight, clip_num):
    """Use Ollama to create engaging titles/hashtags"""
    prompt = f"""
    Create social media metadata for this clip:
    - 1 catchy title (emoji+power words)
    - 5 relevant hashtags
    - 1 call-to-action question
    
    Clip Content: "{highlight['text']}"
    Engagement Reason: "{highlight['reason']}"
    
    Return JSON format:
    {{
        "title": "...",
        "hashtags": ["...", "..."],
        "cta": "..."
    }}
    """
    
    response = ollama.generate(
        model=CONFIG["ollama_model"],
        prompt=prompt,
        format="json"
    )
    
    try:
        metadata = json.loads(response["response"])
    except:
        metadata = {
            "title": f"🔥 {highlight['text'][:50]}...",
            "hashtags": ["#viral", "#trending", "#shorts"],
            "cta": "What do you think?"
        }
    
    # Save metadata
    with open(os.path.join(CONFIG["output_dir"], "clips", f"metadata_{clip_num}.json"), 'w') as f:
        json.dump(metadata, f)
    
    return metadata


           


def main(youtube_url):
    setup_directories()
    
    print("⏬ Downloading video...")
    video_path = download_video(youtube_url)
    if not video_path:
        return
    
    print("🔊 Extracting audio...")
    audio_path = extract_audio(video_path)
    
    print("📝 Transcribing content...")
    transcript = transcribe_with_whisper(audio_path)
    
    print("🤖 Analyzing for highlights...")
    highlights = analyze_content(transcript)
    
    print("🎬 Creating Shorts/Reels...")
    for i, highlight in enumerate(highlights[:5]):
        try:
            print(f"  Creating clip {i+1}: {highlight.get('text', 'Untitled')[:50]}...")
            clip_path = create_vertical_clip(video_path, highlight, i)
            metadata = generate_social_metadata(highlight, i)
            
            print(f"""
            ✅ Clip {i+1} Complete!
            Title: {metadata.get('title', 'Untitled')}
            Hashtags: {' '.join(metadata.get('hashtags', ['#viral']))}
            """)
        except Exception as e:
            print(f"❌ Failed to process clip {i+1}: {e}")
    
    print(f"\n🎉 Created {len(highlights[:5])} clips in {CONFIG['output_dir']}/clips")
if __name__ == "__main__":
    youtube_url = input("Enter YouTube URL: ")
    main(youtube_url)